In [1]:
import numpy as np
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
from fbprophet import Prophet
from typing import Dict
import seaborn as sns
import os
import shutil

# Folder to save results
output_folder = "data/profit_forecast"

if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder)

In [2]:
df = pd.read_csv("data/data.csv").dropna(how="all").drop("id", axis=1).set_index("date")

df.index = pd.to_datetime(df.index, format="%d/%m/%y")

# Create price per ml column
df["container"] = df["container"] + "-" + df["capacity"]
df["capacity"] = df["capacity"].map({"500ml": 500, "1.5lt": 1500, "330ml": 330})
df["price_ml"] = df["price"] / df["capacity"]

# Create profit column
df["profit"] = df["price"] * df["quantity"]

df["profit_per_capita"] = df["profit"] / df["pop"]

# Save the csv 
df.to_csv(os.path.join(output_folder, "raw_data.csv"))

df.shape

(6480, 13)

# EDA

In [3]:
df.isna().sum()

city                  0
lat                  51
long                 46
pop                   0
shop                  0
brand                 0
container            31
capacity             15
price                 0
quantity              0
price_ml             15
profit                0
profit_per_capita     0
dtype: int64

In [4]:
df.head()

,city,lat,long,pop,shop,brand,container,capacity,price,quantity,price_ml,profit,profit_per_capita
date,,,,,,,,,,,,,
2012-01-31,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,glass-500ml,500.0,0.96,13280.0,0.001920,12748.80,0.018968
2012-01-31,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,plastic-1.5lt,1500.0,2.86,6727.0,0.001907,19239.22,0.028624
2012-01-31,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,can-330ml,330.0,0.87,9848.0,0.002636,8567.76,0.012747
2012-01-31,Athens,37.97945,23.71622,672130.0,shop_1,adult-cola,glass-500ml,500.0,1.00,20050.0,0.002000,20050.00,0.029831
2012-01-31,Athens,37.97945,23.71622,672130.0,shop_1,adult-cola,can-330ml,330.0,0.39,25696.0,0.001182,10021.44,0.014910


In [5]:
def plot_profit_per_city(year: int, df: pd.DataFrame):
    """ Plots the profit per city on a given year """
    fig = px.bar(df.loc[df.index.year==year,:].groupby("city", as_index=False)["profit"].sum().sort_values("profit"), 
                 x="city", 
                 y="profit", 
                 title=f"Total Profit per city on {year}",
                 color_discrete_sequence = [f"rgb{sns.color_palette('deep')[0]}"]
                )
    
    fig.layout.yaxis.title="Profit (€)"
    fig.layout.xaxis.title="City"
    return fig

fig = plot_profit_per_city(2016, df)
fig.show() 


In [6]:
def plot_profit_per_capita(year: int, df: pd.DataFrame):
    """ Plots the profit per city on a given year """
    fig = px.bar(df.loc[df.index.year==year,:].groupby("city", as_index=False)["profit_per_capita"].mean().round(2).sort_values("profit_per_capita"), 
                 x="city", 
                 y="profit_per_capita", 
                 title=f"Profit per capita on {year}",
                 color_discrete_sequence = [f"rgb{sns.color_palette('deep')[0]}"]
                )
    
    fig.layout.yaxis.title="Profit per capita (€)"
    fig.layout.xaxis.title="City"
    return fig

fig = plot_profit_per_capita(2017, df)
fig.show()

In [28]:
def plot_profit_per_container(year: int, df: pd.DataFrame):
    """ Plots the profit per container per brand on a given year"""
    
    df_ = df.drop(["lat", "long", "capacity"], axis=1).dropna().copy()
    color_dict = dict(zip(["glass-500ml", 'plastic-1.5lt', 'can-330ml'], [f"rgb{c}" for c in sns.color_palette("deep")]))
    
    fig = px.box(df_[df_.index.year == year].dropna().sort_values(["container", "brand"]), x="brand", color="container", color_discrete_map=color_dict, y="price_ml", title=f"Monthly profit per ml for each brand and container on {year}")
    fig.layout.yaxis.title = "Monthly Profit per ml sold (€/ml)"
    fig.layout.xaxis.title = "Brand"
    return fig

fig = plot_profit_per_container(2017, df)
fig.show()

# Fit the model

In [8]:
%%time
profit_per_city_df = df.reset_index().reset_index().groupby(["date", "city"], as_index=False)["profit"].sum().pivot_table(index="date", columns="city", values="profit").asfreq("m")

def make_forecast(ds: pd.Series) -> pd.DataFrame:
    """ Makes the forecast """
    
    # Prepare the dataframe for prophet
    m = Prophet()
    df = ds.to_frame().reset_index().rename(columns={"date": "ds", ds.name: "y"})
    
    # Fit and create the 12 year forecast
    m.fit(df)
    future = m.make_future_dataframe(periods=12, freq='MS')
    fcst = m.predict(future)
    fcst["city"] = ds.name
    
    # Append the real values
    fcst.set_index("ds", inplace=True)
    fcst["real_values"] = profit_per_city_df[ds.name]
    
    # Plot real values as a solid line and forecasted values as a dashed line
    fcst["forecast"] = fcst["yhat"]
    fcst.loc[~fcst["real_values"].isna(), "forecast"] = np.nan
    
    # Calculate the error
    fcst["error"] = fcst["yhat_upper"] - fcst["yhat"]
    
    return fcst

forecasts = pd.concat(make_forecast(profit_per_city_df[c]) for c in profit_per_city_df.columns)
forecasts.to_csv(os.path.join(output_folder, "forecasts.csv"))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

# Plotting functions
## Plot the forecasts

In [16]:
def plot_cities_forecast(forecasts: Dict[str, pd.DataFrame]):
    """ Creates the plotly figure for the cities forecast """
    
    # Get color scheme from seaborn
    city_color_dictionary = dict(zip(forecasts["city"].unique(), [f"rgb{c}" for c in sns.color_palette("deep")]))


    fig = go.Figure()
    for city in forecasts["city"].unique():

        # Plot real values
        fig.add_trace(go.Scatter(x=forecasts[forecasts["city"]==city].index, 
                                 y=forecasts.loc[forecasts["city"]==city, "real_values"], 
                                 name=city, 
                                 legendgroup=city,
                                 line=dict(color=city_color_dictionary[city]),
                                 mode='lines'))
        # Plot Predictions
        forecasting_df = forecasts.loc[(forecasts["city"]==city) & (forecasts["real_values"].isna()), :]
        fig.add_trace(go.Scatter(x=forecasting_df.index, 
                             y=forecasting_df["forecast"], 
                             name=city, 
                             legendgroup=city,
                             mode='lines+markers',
                             line=dict(color=city_color_dictionary[city], width=1, dash='dot'),
                             error_y=dict(type='data', array=forecasting_df["error"], visible=True),    
                             showlegend=False))
        
    # Add a vertical line to mark the forecasting period
    fig.add_shape(type='line',
                yref="paper",
                xref="x",
                x0=forecasts.loc[forecasts["city"]==city, "real_values"].dropna().index.max(),
                y0=0,
                x1=forecasts.loc[forecasts["city"]==city, "real_values"].dropna().index.max(),
                y1=1,
                line=dict(color='black', width=0.5, dash="dash"))
    
    # Add annoation on the forecast line 
    fig.add_annotation(
                x=forecasts.loc[forecasts["city"]==city, "real_values"].dropna().index.max(),
                y=1,
                yref='paper',
                xanchor="right",
                yanchor="top",
                showarrow=False,
                textangle=-90,
                text="Forecast Start")

    fig.layout.title.text = "Monthly Profit per city."
    fig.layout.yaxis.title = "Profit (€)"
    fig.layout.xaxis.title = "Date"
    fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(legend_title_text='City')
    return fig

plot_cities_forecast(forecasts)